<a href="https://colab.research.google.com/github/jnlinao/NLP/blob/main/Wk4_Ensemble_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import packages
Make sure you installed ***eli5***, ***sklearn***, ***matplotlib*** and ***numpy*** if you use your local machine

In [ ]:
!pip install eli5

     |████████████████████████████████| 106 kB 5.3 MB/s 


In [ ]:
import eli5
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn import datasets
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Compare Logistic Regression and Decision Tree

## Prepare dataset and Pick two classes
Your two classes should be similar, but opposite in some sense

In [ ]:
# categories = ['alt.atheism', 'soc.religion.christian']
categories = ['comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware']
# categories = ['rec.sport.baseball', 'rec.sport.hockey']
# 'alt.atheism','comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware',
# 'comp.sys.mac.hardware','comp.windows.x', 'misc.forsale', 'rec.autos',  
# 'rec.motorcycles',  'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
# 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns',
# 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc'
train = sklearn.datasets.fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'),)
test = sklearn.datasets.fetch_20newsgroups(subset='test', categories=categories, remove=('headers', 'footers', 'quotes'),)
print('train data size:', len(train.data))
print('test data size:', len(test.data))

train data size: 1168
test data size: 777


## Compare Logistic Regression and Decision Tree models


In [ ]:
lr_model = LogisticRegression(C=1, solver='newton-cg')
lr_features = CountVectorizer() 
lr_classifier = make_pipeline(lr_features, lr_model)
lr_classifier.fit(train.data, train.target)

dt_model = DecisionTreeClassifier(min_samples_split=0.4)
dt_features = CountVectorizer()
dt_classifier = make_pipeline(dt_features, dt_model)
dt_classifier.fit(train.data, train.target)

#Compare accuracy of the two models
lr_train_preds = lr_classifier.predict(train.data)
lr_train_f1 = f1_score(train.target, lr_train_preds, average='micro')
lr_test_preds = lr_classifier.predict(test.data)
lr_test_f1 = f1_score(test.target, lr_test_preds, average='micro')
print("Train/test F1 for Logistic Regression: ", lr_train_f1, lr_test_f1)

dt_train_preds = dt_classifier.predict(train.data)
dt_train_f1 = f1_score(train.target, dt_train_preds, average='micro')
dt_test_preds = dt_classifier.predict(test.data)
dt_test_f1 = f1_score(test.target, dt_test_preds, average='micro')
print("Train/test F1 for Decision Tree: ", dt_train_f1, dt_test_f1)

Train/test F1 for Logistic Regression:  0.9897260273972602 0.8095238095238095
Train/test F1 for Decision Tree:  0.7722602739726028 0.7400257400257402


In [ ]:
eli5.show_weights(lr_classifier, top=20, target_names=test.target_names)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
eli5.show_weights(dt_classifier, top=5, target_names=test.target_names)
#Play with the min_samples_split parameter when creating dt_classifier and see how the tree changes
#TODO FOR STUDENT: What happens when to the tree when you modify the min_samples_split_parameter

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
idx = 2
x = test.data[idx]
print(x)


[REMAINDER DELETED]

I don't have my copy of the manual with me right now, but I can offer the
following in the interim:

   1)  The card uses port addresses 0x2E0 and 0x2E8 (which are NOT
       configurable).  These addresses, incidentally, were inadvertantly
       omitted from my version of the manual.

   2)  I believe there is a dip that controls whether or not to enable
       IRQ 2 (for CGA or EGA support??!?).

Lance Hartmann (lance%hartmann.austin.ibm.com@ibmpa.awdpa.ibm.com)
               Yes, that IS a '%' (percent sign) in my network address.


In [ ]:
idx = 2
x = test.data[idx]
print(test.target_names[test.target[idx]])
eli5.show_prediction(lr_model, test.data[idx], vec=lr_features, target_names=test.target_names)

comp.sys.ibm.pc.hardware


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
eli5.show_prediction(dt_model, test.data[idx], vec=dt_features, target_names=test.target_names)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Ensemble Methods

In [ ]:
from sklearn.ensemble import VotingClassifier

features = CountVectorizer()

lr_model = LogisticRegression(C=1, solver='lbfgs')
lr_classifier = make_pipeline(features, lr_model)
lr_classifier.fit(train.data, train.target)

#TODO FOR STUDENT: Try playing with the min_samples_split to see how it affect the ensemble score
dt_model = DecisionTreeClassifier(min_samples_split=0.35)
dt_classifier = make_pipeline(features, dt_model)
dt_classifier.fit(train.data, train.target)

#Compare accuracy of the two models
lr_train_preds = lr_classifier.predict(train.data)
lr_train_f1 = f1_score(train.target, lr_train_preds, average='micro')
lr_test_preds = lr_classifier.predict(test.data)
lr_test_f1 = f1_score(test.target, lr_test_preds, average='micro')
print("Train/test F1 for Logistic Regression: ", lr_train_f1, lr_test_f1)

dt_train_preds = dt_classifier.predict(train.data)
dt_train_f1 = f1_score(train.target, dt_train_preds, average='micro')
dt_test_preds = dt_classifier.predict(test.data)
dt_test_f1 = f1_score(test.target, dt_test_preds, average='micro')
print("Train/test F1 for Decision Tree: ", dt_train_f1, dt_test_f1)

#Look at classifier agreement
print("\n% Cases where the two classifiers agree on test data: ", np.sum(lr_test_preds == dt_test_preds)/len(lr_test_preds))
print("% Cases where one of the two classifiers has correct answer: ", np.sum(np.logical_or(lr_test_preds == test.target, dt_test_preds == test.target)/len(lr_test_preds)))

#Try to build an ensemble combing both models
#TODO FOR STUDENT: Modify the weights parameter which give different weight to each of the classifiers
ensemble_classifier = make_pipeline(lr_features, VotingClassifier(estimators=[('lr', lr_model), ('dt', dt_model)], voting='soft', weights=[4,3.5]))
ensemble_classifier.fit(train.data, train.target) 

ensemble_train_preds = ensemble_classifier.predict(train.data)
ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
ensemble_test_preds = ensemble_classifier.predict(test.data)
ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
print("\nTrain/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)


Train/test F1 for Logistic Regression:  0.9897260273972602 0.8082368082368082
Train/test F1 for Decision Tree:  0.803082191780822 0.7747747747747747

% Cases where the two classifiers agree on test data:  0.7786357786357786
% Cases where one of the two classifiers has correct answer:  0.9021879021879022

Train/test F1 for Ensemble:  0.9828767123287672 0.8455598455598455


## Bagging

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#TODO FOR STUDENT: Try playing with n_estimators and min_samples_split
ensemble_classifier = make_pipeline(lr_features, RandomForestClassifier(n_estimators=50, min_samples_split=0.25))
ensemble_classifier.fit(train.data, train.target)

ensemble_train_preds = ensemble_classifier.predict(train.data)
ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
ensemble_test_preds = ensemble_classifier.predict(test.data)
ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
print("\nTrain/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)



Train/test F1 for Ensemble:  0.9297945205479452 0.8198198198198198


## Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ensemble_classifier = make_pipeline(lr_features, AdaBoostClassifier(n_estimators=50, learning_rate=0.2))
ensemble_classifier.fit(train.data, train.target)

ensemble_train_preds = ensemble_classifier.predict(train.data)
ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
ensemble_test_preds = ensemble_classifier.predict(test.data)
ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
print("\nTrain/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)



Train/test F1 for Ensemble:  0.788527397260274 0.7760617760617761


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

#TODO FOR STUDENT: Try playing with n_estimators and min_samples_split
ensemble_classifier = make_pipeline(lr_features, GradientBoostingClassifier(n_estimators=50, min_samples_split=0.25))
ensemble_classifier.fit(train.data, train.target)

ensemble_train_preds = ensemble_classifier.predict(train.data)
ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
ensemble_test_preds = ensemble_classifier.predict(test.data)
ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
print("\nTrain/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)



Train/test F1 for Ensemble:  0.8210616438356164 0.7850707850707851


In [ ]:
from xgboost import XGBClassifier

#TODO FOR STUDENT: Try playing with n_estimators and min_samples_split
ensemble_classifier = make_pipeline(lr_features, XGBClassifier(n_estimators=50, max_depth=8))
ensemble_classifier.fit(train.data, train.target)

ensemble_train_preds = ensemble_classifier.predict(train.data)
ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
ensemble_test_preds = ensemble_classifier.predict(test.data)
ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
print("\nTrain/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)





Train/test F1 for Ensemble:  0.928082191780822 0.8108108108108109


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

#TODO FOR STUDENT: Try playing with n_estimators and min_samples_split
ensemble_classifier = make_pipeline(lr_features, GradientBoostingClassifier(n_estimators=100, min_samples_split=0.2))
ensemble_classifier.fit(train.data, train.target)

ensemble_train_preds = ensemble_classifier.predict(train.data)
ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
ensemble_test_preds = ensemble_classifier.predict(test.data)
ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
print("\nTrain/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)


# Comparing Bagging and Boosting

In [ ]:
for n_est in range(50,500,50):
  ensemble_classifier = make_pipeline(lr_features, RandomForestClassifier(n_estimators=n_est, min_samples_split=0.008))
  ensemble_classifier.fit(train.data, train.target)

  ensemble_train_preds = ensemble_classifier.predict(train.data)
  ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
  ensemble_test_preds = ensemble_classifier.predict(test.data)
  ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
  print(n_est, "Train/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)

50 Train/test F1 for Ensemble:  0.9897260273972602 0.8198198198198198
100 Train/test F1 for Ensemble:  0.9888698630136986 0.8326898326898327
150 Train/test F1 for Ensemble:  0.9888698630136986 0.8198198198198198
200 Train/test F1 for Ensemble:  0.9897260273972602 0.8326898326898327
250 Train/test F1 for Ensemble:  0.9897260273972602 0.8352638352638352
300 Train/test F1 for Ensemble:  0.9897260273972602 0.8442728442728443
350 Train/test F1 for Ensemble:  0.9888698630136986 0.8378378378378378
400 Train/test F1 for Ensemble:  0.9897260273972602 0.8404118404118404
450 Train/test F1 for Ensemble:  0.9897260273972602 0.8352638352638352


In [ ]:
for n_est in range(50,501,50):
  ensemble_classifier = make_pipeline(lr_features, RandomForestClassifier(n_estimators=n_est, min_samples_split=0.25))
  ensemble_classifier.fit(train.data, train.target)

  ensemble_train_preds = ensemble_classifier.predict(train.data)
  ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
  ensemble_test_preds = ensemble_classifier.predict(test.data)
  ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
  print(n_est, "Train/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)

50 Train/test F1 for Ensemble:  0.922945205479452 0.8043758043758044
100 Train/test F1 for Ensemble:  0.9332191780821918 0.8288288288288288
150 Train/test F1 for Ensemble:  0.9238013698630136 0.8378378378378378
200 Train/test F1 for Ensemble:  0.9272260273972602 0.824967824967825
250 Train/test F1 for Ensemble:  0.928082191780822 0.8314028314028314
300 Train/test F1 for Ensemble:  0.9392123287671232 0.8288288288288288
350 Train/test F1 for Ensemble:  0.9306506849315068 0.8404118404118404
400 Train/test F1 for Ensemble:  0.9297945205479452 0.8326898326898327
450 Train/test F1 for Ensemble:  0.9332191780821918 0.8378378378378378
500 Train/test F1 for Ensemble:  0.934931506849315 0.8314028314028314


In [ ]:
for n_est in range(50,500,50):
  ensemble_classifier = make_pipeline(lr_features, GradientBoostingClassifier(n_estimators=n_est, min_samples_split=0.05))
  ensemble_classifier.fit(train.data, train.target)

  ensemble_train_preds = ensemble_classifier.predict(train.data)
  ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
  ensemble_test_preds = ensemble_classifier.predict(test.data)
  ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
  print(n_est, "Train/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)

50 Train/test F1 for Ensemble:  0.8467465753424658 0.7876447876447877
100 Train/test F1 for Ensemble:  0.9238013698630136 0.8301158301158301
150 Train/test F1 for Ensemble:  0.9529109589041096 0.8301158301158301
200 Train/test F1 for Ensemble:  0.9674657534246576 0.833976833976834
250 Train/test F1 for Ensemble:  0.9845890410958904 0.824967824967825
300 Train/test F1 for Ensemble:  0.985445205479452 0.833976833976834
350 Train/test F1 for Ensemble:  0.985445205479452 0.8391248391248392
400 Train/test F1 for Ensemble:  0.9845890410958904 0.8494208494208494
450 Train/test F1 for Ensemble:  0.9863013698630136 0.842985842985843


In [ ]:
for n_est in range(50,500,50):
  ensemble_classifier = make_pipeline(lr_features, GradientBoostingClassifier(n_estimators=n_est, min_samples_split=0.5))
  ensemble_classifier.fit(train.data, train.target)

  ensemble_train_preds = ensemble_classifier.predict(train.data)
  ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
  ensemble_test_preds = ensemble_classifier.predict(test.data)
  ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
  print(n_est, "Train/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)

50 Train/test F1 for Ensemble:  0.8210616438356164 0.7850707850707851
100 Train/test F1 for Ensemble:  0.877568493150685 0.8095238095238095
150 Train/test F1 for Ensemble:  0.9375 0.8301158301158301
200 Train/test F1 for Ensemble:  0.9511986301369864 0.8416988416988417
250 Train/test F1 for Ensemble:  0.9811643835616438 0.8301158301158301
300 Train/test F1 for Ensemble:  0.9828767123287672 0.8365508365508365
350 Train/test F1 for Ensemble:  0.9845890410958904 0.8416988416988417
400 Train/test F1 for Ensemble:  0.9837328767123288 0.8314028314028314
450 Train/test F1 for Ensemble:  0.985445205479452 0.8365508365508365


In [ ]:
for n_est in range(50,500,50):
  ensemble_classifier = make_pipeline(lr_features, GradientBoostingClassifier(n_estimators=n_est, min_samples_split=0.05))
  ensemble_classifier.fit(train.data, train.target)

  ensemble_train_preds = ensemble_classifier.predict(train.data)
  ensemble_train_f1 = f1_score(train.target, ensemble_train_preds, average='micro')
  ensemble_test_preds = ensemble_classifier.predict(test.data)
  ensemble_test_f1 = f1_score(test.target, ensemble_test_preds, average='micro')
  print(n_est, "Train/test F1 for Ensemble: ", ensemble_train_f1, ensemble_test_f1)